In [38]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
from recommendation_engine import *
import ipywidgets as widgets
from ipywidgets import interactive,Layout


feature_df = pd.read_csv('song_features.csv',index_col=0)
model = Doc2Vec.load('doc2vec_len100.model')
decode_df = pd.read_csv('song_ids_full.txt',sep='\t')
decode_dict = decoder(decode_df)

artists = sorted(decode_df['artist_name'].unique().tolist())
songs = sorted(decode_df['song_title'].tolist())

Initialize some of our parameters.

In [39]:
"""These weight the relative importance we put on each in determining similarity. 

feature_importance is the weight we put on similarities found from our extracted
features (syllables per second, sentiment,etc).

doc_importance is the weight we put on the similarities found from our doc2vec model.
"""

feature_importance=0.5
doc_importance = 0.5

In [23]:
def view_artist(x=''):
    if x=='': return 
    else: return decode_df[(decode_df.artist_name==x)]
    

    

artist_sel = widgets.Select(options=artists)
artist_sel.layout.height='300px'
interactive(view_artist, x=artist_sel)

#song_sel = widgets.Select(options=decode_df[decode_df.artist_name==artist_sel.value]['song_title'].to_list())

interactive(children=(Select(description='x', layout=Layout(height='300px'), options=('03 Greedo', '070 Shake'…

After selecting an artist from above, rerun the cell below to see their available songs. Hopefully I can get these widgets linked together soon. Linking at the moment was crashing jupyter.

In [50]:
def view_song(x=''):
    if x=='': return 
    else: return decode_df[(decode_df.song_title==x)]
    
song_sel = widgets.Select(options=decode_df[(decode_df.artist_name==artist_sel.value)]['song_title'].to_list())
song_sel.layout.height='300px'
interactive(view_song, x=song_sel)

interactive(children=(Select(description='x', layout=Layout(height='300px'), options=('1017 Lifestyle', '10 Ye…

In [51]:
song_id = decode_df[(decode_df.artist_name==artist_sel.value)&(decode_df.song_title==song_sel.value)]['song_id'].values[0]
feature_df = clean_features(feature_df)
similarities1 = feature_similarity(song_id,feature_df)
similarities2 = similarity_df(similarities1,model,song_id,feature_importance=feature_importance,doc_importance=doc_importance)
recommend(similarities2,decode_dict,song_id)

Recommendations for Do U Love Me? by Young Thug

Try out 'Feel Good' by Lil Durk
Try out 'Ay Ya Ya Ya' by Yella Beezy
Try out 'Wet (She Got That…)' by YFN Lucci
Try out 'You in Luv wit Her' by 2 Chainz
Try out 'All Nighter' by Doja Cat


In [2]:
song_id = 1000113
feature_df = clean_features(feature_df)
similarities1 = feature_similarity(song_id,feature_df)
similarities2 = similarity_df(similarities1,model,song_id)
recommend(similarities2,decode_dict,song_id)

Recommendations for Let It Go by Ray J

Try out Come Dance With Me by Sleepy Brown
Try out Hump Bounce by R. Kelly
Try out Dance With Me by Sleepy Brown
Try out Come Baby Come (Ultimix) by K7
Try out Afrika shox by Afrika Bambaataa
